**Rakshit Vasava, Individual Assignment 3**

In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment.
# For the code to do so you can search “mount” in code snippets or use the code given below.
# Our entire drive contents are now mounted on colab at the location “/gdrive”.
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
!pip install vecstack

In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter #for Smote,

import warnings
warnings.filterwarnings("ignore")

In [ ]:
trainfile = r'/gdrive/My Drive/Colab Notebooks/RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/Colab Notebooks/RevisedHomesiteTest1.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(test_data.shape)
print(train_data.tail())
print(test_data.tail())

(65000, 596)
(173836, 596)
       CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
64995                 3                 5                9               15   
64996                 5                 9               14               21   
64997                 7                14                3                2   
64998                 5                 9               15               22   
64999                 4                 6                7               12   

       CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
64995                9               15                8               13   
64996               14               21               13               20   
64997                3                3                2                2   
64998               15               22               13               21   
64999                8               13                7               11   

       CoverageField4A  CoverageFie

In [ ]:
#Organize Train data and Test data
X_train = train_data.iloc[:, :-1].copy()
Y_train = train_data["QuoteConversion_Flag"]

X_test = test_data.iloc[:, :-1].copy()

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(X_train.tail())
print(Y_train)
print(X_test.tail())

(65000, 595)
(65000,)
(173836, 595)
       CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
64995                 3                 5                9               15   
64996                 5                 9               14               21   
64997                 7                14                3                2   
64998                 5                 9               15               22   
64999                 4                 6                7               12   

       CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
64995                9               15                8               13   
64996               14               21               13               20   
64997                3                3                2                2   
64998               15               22               13               21   
64999                8               13                7               11   

       CoverageField4A  Co

In [ ]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY
clf = DecisionTreeClassifier()
clf.fit(X_train,Y_train)
clf_predict=clf.predict(X_test)
# print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_test1,y_test)))
# print("Confusion Matrix for Decision Tree")
# print(confusion_matrix(y_test,clf_predict))

In [ ]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, Y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,Y_train)
clf_predict = clf.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
# print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_test1,y_test)))
# print("Confusion Matrix after hypertuning for Decision Tree")
# print(confusion_matrix(y_test,clf_predict))
# print("=== Classification Report ===")
# print(classification_report(y_test,clf_predict))

#get cross-validation report
clf_cv_score = cross_val_score(clf, X_train, Y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 50, 'max_depth': 7}
=== All AUC Scores ===
[0.93940456 0.93388671 0.92640534 0.93591906 0.9308566  0.9374214
 0.93840122 0.9281893  0.93865823 0.92668027]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9335822687537098


In [ ]:
#Construct Random Forest Model

rfc = RandomForestClassifier()
rfc.fit(X_train, Y_train)
rfc_predict=rfc.predict(X_test)
# print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_test1,y_test)))
# print("Confusion Matrix for Random Forest:")
# print(confusion_matrix(y_test,rfc_predict))

In [ ]:
#Construct MultiLayer Perceptron Model
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=100)
mlp.fit(X_train, Y_train)
mlp_predict=mlp.predict(X_test)
# print("accuracy Score (training) for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test1,y_test)))
# print("Confusion Matrix for MultiLayer Perceptron:")
# print(confusion_matrix(y_test,mlp_predict))

In [ ]:
#Construct K-Nearest Neighbor Model
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(X_train, Y_train)
neigh_predict=neigh.predict(X_test)
# print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test1,y_test)))
# print("Confusion Matrix for KNeighborsClassifier:")
# print(confusion_matrix(y_test,neigh_predict))

In [ ]:
#Construct Linear Support Vector Machine Model
from sklearn.svm import LinearSVC
linsvm = LinearSVC(max_iter=300)
linsvm.fit(X_train, Y_train)
linsvm_predict=linsvm.predict(X_test)
# print("accuracy Score (training) for Linear SVM Classifier:{0:6f}".format(linsvm.score(X_test1,y_test)))
# print("Confusion Matrix for Linear SVM Classifier:")
# print(confusion_matrix(y_test,linsvm_predict))

In [ ]:
#Construct Gradient Boosting model

search_grid={'n_estimators':[5,10,20],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train, Y_train)
abc_predict=abc.predict(X_test)
# print("accuracy Score (training) for Boosting:{0:6f}".format(abc.score(X_test1,y_test)))
# print("Confusion Matrix for boosting:")
# print(confusion_matrix(y_test,abc_predict))

abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train, Y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train,Y_train)
abc_predict = abc.predict(X_test)
# print("accuracy Score (training) after hypertuning for Boosting:{0:6f}".format(abc.score(X_test1,y_test)))
# print("Confusion Matrix after hypertuning for Boosting:")
# print(confusion_matrix(y_test,abc_predict))
# print("=== Classification Report ===")
# print(classification_report(y_test,abc_predict))

abc_cv_score = cross_val_score(abc, X_train, Y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())

{'n_estimators': 20, 'learning_rate': 0.1}
=== All AUC Scores ===
[0.93884772 0.92937421 0.9293074  0.93389785 0.92874445 0.94154749
 0.94628803 0.9207169  0.93271786 0.92644673]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  0.9327888629821928


**SMOTE**

In [ ]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(Y_train))
sm = SMOTE(sampling_strategy=0.5)
X_res, y_res = sm.fit_resample(X_train, Y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


**Stacking**

In [ ]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ KNeighborsClassifier(), MLPClassifier(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]

S_Train, S_Test = stacking(models,
                           X_res, y_res, X_test,
                           regression=False,

                           mode='oof_pred_bag',

                           needs_proba=False,

                           save_dir=None,

                           metric=accuracy_score,

                           n_folds=4,

                           stratified=True,

                           shuffle=True,

                           random_state=0,

                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.70314001]
    fold  1:  [0.69727461]
    fold  2:  [0.69909491]
    fold  3:  [0.69559061]
    ----
    MEAN:     [0.69877504] + [0.00280834]
    FULL:     [0.69877508]

model  1:     [MLPClassifier]
    fold  0:  [0.84623553]
    fold  1:  [0.84709511]
    fold  2:  [0.89523183]
    fold  3:  [0.84167678]
    ----
    MEAN:     [0.85755981] + [0.02184720]
    FULL:     [0.85756001]

model  2:     [LinearSVC]
    fold  0:  [0.67679628]
    fold  1:  [0.68518987]
    fold  2:  [0.58633766]
    fold  3:  [0.58328277]
    ----
    MEAN:     [0.63290164] + [0.04819501]
    FULL:     [0.63290227]

model  3:     [RandomForestCla

In [ ]:
model = RandomForestClassifier()

model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)

In [ ]:
print(y_pred)

[0 0 0 ... 1 0 0]


In [ ]:
S_Test_df = pd.DataFrame(S_Test)
y_pred_df = pd.DataFrame(y_pred)

print(S_Test_df.shape)
print(y_pred_df.shape)
print(S_Test_df.head())
print(y_pred_df.head())

(173836, 5)
(173836, 1)
   0  1  2  3  4
0  0  0  0  0  0
1  0  0  0  0  0
2  1  0  0  0  0
3  0  0  0  0  0
4  0  0  0  0  0
   0
0  0
1  0
2  0
3  0
4  0


In [ ]:
model.predict_proba(S_Test)

# '/gdrive/My Drive/Colab Notebooks/sample_submission.csv'
# Export as CSV
sub=pd.read_csv("/gdrive/My Drive/Colab Notebooks/sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(y_pred,columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("/gdrive/My Drive/Colab Notebooks/CIS508_Assg3_Result.csv",index=False)

**Hyperparameter Tuning for Stacking**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from vecstack import stacking
from scipy.stats import randint

# Define base models
models = [
    KNeighborsClassifier(),
    MLPClassifier(),
    LinearSVC(),
    RandomForestClassifier(),
    DecisionTreeClassifier()
]

# Fit each model and create predictions
def create_base_predictions(models, X_train, X_test, y_train):
    meta_X_train = []
    meta_X_test = []
    for model in models:
        model.fit(X_train, y_train)
        meta_X_train.append(model.predict(X_train))
        meta_X_test.append(model.predict(X_test))
    return np.column_stack(meta_X_train), np.column_stack(meta_X_test)

# Assuming X_res, y_res, X_test are your training, validation, and testing sets respectively
# Get predictions from base models
S_train, S_test = create_base_predictions(models, X_res, X_test, y_res)

# Define parameters for meta-classifier (Random Forest in this case)
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': [10, 20, 30, 40, None],
    # Add other parameters for RandomForestClassifier as needed
}

# Instantiate Random Forest for meta-classifier
meta_model = RandomForestClassifier()

# Perform RandomizedSearchCV for meta-classifier
random_search = RandomizedSearchCV(meta_model, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', verbose=2)
random_search.fit(S_train, y_res)

# Get best parameters and best estimator
best_params = random_search.best_params_
best_estimator = random_search.best_estimator_

# Predict using the best estimator for the meta-classifier
final_predictions = best_estimator.predict(S_test)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ......................max_depth=30, n_estimators=70; total time=   1.0s
[CV] END ......................max_depth=30, n_estimators=70; total time=   1.0s
[CV] END ......................max_depth=30, n_estimators=70; total time=   1.0s
[CV] END .....................max_depth=20, n_estimators=177; total time=   2.5s
[CV] END .....................max_depth=20, n_estimators=177; total time=   1.7s
[CV] END .....................max_depth=20, n_estimators=177; total time=   1.7s
[CV] END ......................max_depth=40, n_estimators=96; total time=   0.9s
[CV] END ......................max_depth=40, n_estimators=96; total time=   0.9s
[CV] END ......................max_depth=40, n_estimators=96; total time=   1.0s
[CV] END ...................max_depth=None, n_estimators=134; total time=   1.2s
[CV] END ...................max_depth=None, n_estimators=134; total time=   1.3s
[CV] END ...................max_depth=None, n_es

In [ ]:
print(final_predictions)

[0 0 1 ... 1 0 1]


In [ ]:
sub=pd.read_csv("/gdrive/My Drive/Colab Notebooks/sample_submission.csv")
output=pd.concat([sub["QuoteNumber"],pd.DataFrame(final_predictions,columns=["QuoteConversion_Flag"])],axis=1)
output.to_csv("/gdrive/My Drive/Colab Notebooks/CIS508_Assg3_Result_1.csv",index=False)